# 02d Paralelismo

El paralelismo es la capacidad de ejecutar varias tareas simultáneamente, aprovechando que la mayoría de ordenadores actuales tienen una arquitectura de varios núcleos. Hay dos modelos principales de paralelismo:

* **Paralelismo por procesos:** Hay varias copias del mismo programa (proceso) ejecutándose a la vez, de modo que si abres el Administrador de tareas, verás que `Python` aparece varias veces.
* **Paralelismo por "hilos" (threads):** Hay un único proceso, que tiene varios conjuntos de código (threads) ejecutándose a la vez.

En general, el paralelismo por procesos es mejor para situaciones que requieren mucho uso de CPU, como cálculos matemáticos, mientras que el paralelismo por threads es mejor cuando la ejecución está limitada por operaciones de lectura y escritura. Además, la implementación estándar de Python, CPython, limita el número de threads ejecutándose simultáneamente a 1 (GIL), por lo que el paralelismo por threads no ofrece absolutamente ninguna ventaja en tareas de CPU.

In [1]:
# Código para importar Temporizador desde utils.py, no es importante

import os, sys
dir2 = os.path.abspath('')
dir1 = os.path.dirname(dir2)
if not dir1 in sys.path: sys.path.append(dir1)
from utils import Temporizador

## Paralelismo por hilos

El paralelismo por hilos está implementado por el módulo `threading` de la librería estándar. Veremos solamente los conceptos más básicos, ya que en general nos interesarán más las tareas limitadas por CPU.

In [36]:
import threading 
from time import sleep

def f(x):
    sleep(5)
    print(x)

t1 = threading.Thread(target=f, args=("Thread 1\n",))
t2 = threading.Thread(target=f, args=("Thread 2\n",))

with Temporizador() as temp:
    t1.start()
    t2.start()
    t1.join()
    t2.join()

print(f"Tiempo total: {temp.ver_tiempo():.4f} s")

Thread 1

Thread 2

Tiempo total: 5.0053 s


Cada thread se crea como un objeto `threading.Thread`, donde `target` indica la función que se va a ejecutar, y `args` es una tupla con sus argumentos. El método `start()` inicia la ejecución del thread, y `join()` espera a que acabe. En este caso, al no tratarse de una tarea que esté limitada por CPU, el tiempo de ejecución se ha visto reducido.

En cambio, veamos qué ocurre con un cálculo matemático, calculando el factorial de los números entre 1000 y 2000:

In [31]:
def factorial(x):
    f = 1
    i = x
    while i > 0:
        f *= i
        i -=1
    return f

def tarea(inicio, fin, lista):
    for x in range(inicio, fin):
        lista[x-1000] = factorial(x)

In [26]:
resultado1 = [0,]*1000

with Temporizador() as temp:
    tarea(1000, 2000, resultado1)

print(f"Tiempo en un único thread: {temp.ver_tiempo():.4f} s")

Tiempo en un único thread: 0.3750 s


In [32]:
resultado2 = [0,]*1000
t1 = threading.Thread(target=tarea, args=(1000, 1500, resultado2))
t2 = threading.Thread(target=tarea, args=(1500, 2000, resultado2))

with Temporizador() as temp:
    t1.start()
    t2.start()
    t1.join()
    t2.join()

print(f"Tiempo en dos threads: {temp.ver_tiempo():.4f} s")

Tiempo en dos threads: 0.4497 s


In [34]:
for i in range(1000):
    if resultado1[i] != resultado2[i]:
        print(f"Error en el elemento {i}")
        break

El tiempo de ejecución es ligeramente mayor usando dos threads que uno solo! Además, no es posible obtener el valor de un `return`, por eso hemos tenido que pasar la lista "por referencia" para almacenar los valores. En este caso sencillo, cada thread escribía en elementos distintos, y no había posibilidad de que intentarán competir entre ellos. Pero en general, esto es una posibilidad que se puede evitar si uno de los threads bloquea temporalmente (lock) la ejecución del resto cuando tiene que usar un recurso compartido.

Finalmente, veamos que ambos threads comparten el mismo proceso. Una forma de hacerlo es abriendo el administrador de tareas, pero también podemos hacerlo desde python: el comando `os.getpid()` devuelve un identificador que es único para cada proceso:

In [35]:
def pid(x):
    sleep(3)
    print(f"Thread {x}, PID: {os.getpid()}\n")

t1 = threading.Thread(target=pid, args=(1,))
t2 = threading.Thread(target=pid, args=(2,))

t1.start()
t2.start()
t1.join()
t2.join()

Thread 1, PID: 13125
Thread 2, PID: 13125


